In [1]:
import numpy as np
from fxpmath import Fxp

In [20]:
class myfi(np.ndarray):
    def __new__(cls, input_array, signed=1, w=32, f=16, rounding='nearest', overflow='wrap', like=None): 
        ndarray = np.asarray(input_array).view(cls)   
        if isinstance(like, myfi):
            ndarray.config(like.rounding, like.overflow)
            ndarray.resize(like.signed, like.w, like.f)            
        else:     
            ndarray.config(rounding, overflow)                         
            ndarray.resize(signed, w, f)            
        return ndarray

    def __getitem__(self, key):
        # let single element index of array return myfi with shape (1,) instead of base dtype element
        return myfi(super().__getitem__(key),like=self)

    def resize(self, signed, w, f):
        self.signed = signed
        self.w = w
        self.f = f
        self.precision = 2**-self.f
        self.upper = 2**(self.w - self.f - (1 if self.signed else 0)) - self.precision 
        self.lower = -2**(self.w-self.f-1) if self.signed else 0     
        self._quantize()

    def config(self, rounding, overflow):
        self.rounding = rounding
        self.overflow = overflow

    def _quantize(self):        
        # TODO: unsigned
        b = self * (2**self.f)
        if self.rounding == 'nearest':
            store_int = np.round(b)
        elif self.rounding == 'floor':
            store_int = np.floor(b)
        else:
            raise ValueError(f'not support rounding method {self.rounding}')
        self[...] = store_int*self.precision # this will only reassgin array value, instead of create new myfi instance

    @property
    def int(self):
        return (np.asarray(self)*2**self.f).astype(int) # return normal ndarray

    def base_repr(self, base=2, frac_point=False): # return ndarray with same shape and dtype = '<Uw' where w=self.w
        if base == 2:
            add_point = lambda s: s[:-self.f] + '.' + s[-self.f:] if frac_point else s
            func = lambda i: add_point(np.binary_repr(i,width=self.w))    
        else:
            func = lambda i: np.base_repr(i,base=base)
        return np.vectorize(func)(self.int)
    @property
    def bin(self): 
        return self.base_repr(2)
    @property 
    def bin_(self):
        return self.base_repr(2,frac_point=True)
    @property
    def hex(self):
        return self.base_repr(16)
        

    #TODO: __add__ __radd__ __iadd__ sub/mul/div/ge/gr/le/etc...
    #TODO: over/underflow
    #TODO: casting


x = np.zeros((3,2,2))
y = myfi(x,1,14,3) # call __new__ and __array_finalize__
print(y.w,y.f,y.precision)
z = y[1:]
print(z.w,z.f,z.precision)
q = myfi(x,like=y)
print(q.w,q.f,q.precision)
w = y[1,1,1]
print(w.w,w.f,w.precision)
# y[1,1,1] # call __array_finalize__
# x.view(myfi) # call __array_fianlize__

14 3 0.125
14 3 0.125
14 3 0.125
14 3 0.125


In [21]:
myfi([3.618],1,13,3)

myfi([3.625])